# Stock NeurIPS2018 Part 3. Backtest
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the third and last part of the NeurIPS2018 series, introducing how to use use the agents we trained to do backtest, and compare with baselines such as Mean Variance Optimization and DJIA index.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages (Passed)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3

import os
import sys
mypath = "/home/lqs/FinRL/"
sys.path.insert(0, mypath)


from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.config import *
from finrl.main import check_and_make_directories
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.meta.env_stock_trading.env_stocktrading_Short import StockTradingEnv_Short
from finrl.meta.env_stock_trading.env_stocktrading_newReward import StockTradingEnv_NewReward
from finrl.meta.env_stock_trading.env_stocktrading_Short_newReward import StockTradingEnv_Short_NewReward


from finrl.meta.preprocessor.yahoodownloader import YahooDownloader



import importlib
import finrl
importlib.reload(finrl)

exp = 'Custom3Action'
# exp = 'Baseline'
# exp = 'CustomRewardFunction'
# exp = 'Reward&3Action'
total_timesteps = 4000


%matplotlib inline

# Part 2. Backtesting

To backtest the agents, upload trade_data.csv in the same directory of this notebook. For Colab users, just upload trade_data.csv to the default directory.

In [2]:
train = pd.read_csv('train_data.csv')
trade = pd.read_csv('trade_data.csv')

# If you are not using the data generated from part 1 of this tutorial, make sure 
# it has the columns and index in the form that could be make into the environment. 
# Then you can comment and skip the following lines.
train = train.set_index(train.columns[0])
train.index.names = ['']
trade = trade.set_index(trade.columns[0])
trade.index.names = ['']

Then, upload the trained agent to the same directory, and set the corresponding variable to True.

In [3]:
if_using_a2c = True
if_using_ddpg = True
if_using_ppo = True
if_using_td3 = True
if_using_sac = True

Load the agents

In [ ]:
trained_a2c = A2C.load(EXP_NAME[exp]+TRAINED_MODEL_DIR + "/agent_a2c") if if_using_a2c else None
trained_ddpg = DDPG.load(EXP_NAME[exp]+TRAINED_MODEL_DIR + "/agent_ddpg") if if_using_ddpg else None
trained_ppo = PPO.load(EXP_NAME[exp]+TRAINED_MODEL_DIR + "/agent_ppo") if if_using_ppo else None
trained_td3 = TD3.load(EXP_NAME[exp]+TRAINED_MODEL_DIR + "/agent_td3") if if_using_td3 else None
trained_sac = SAC.load(EXP_NAME[exp]+TRAINED_MODEL_DIR + "/agent_sac") if if_using_sac else None

### Trading (Out-of-sample Performance)

We update periodically in order to take full advantage of the data, e.g., retrain quarterly, monthly or weekly. We also tune the parameters along the way, in this notebook we use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [ ]:
stock_dimension = len(trade.tic.unique())
state_space = 1 + 2 * stock_dimension + len(INDICATORS) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

In [6]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
}

In [ ]:
e_train_gym = None
if exp == 'Baseline':
    e_trade_gym = StockTradingEnv(df=trade, **env_kwargs)
elif exp == 'Custom3Action':
    e_trade_gym = StockTradingEnv_Short(df=trade, **env_kwargs)
elif exp == 'CustomRewardFunction':
    e_trade_gym = StockTradingEnv_NewReward(df=trade, **env_kwargs)
elif exp == 'Reward&3Action':
    e_trade_gym = StockTradingEnv_Short_NewReward(df=trade, **env_kwargs)

In [ ]:
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c, 
    environment = e_trade_gym) if if_using_a2c else (None, None)

In [ ]:
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym) if if_using_ddpg else (None, None)

In [ ]:
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_ppo, 
    environment = e_trade_gym) if if_using_ppo else (None, None)

In [ ]:
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_td3, 
    environment = e_trade_gym) if if_using_td3 else (None, None)

In [ ]:
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym) if if_using_sac else (None, None)

In [ ]:
# Calculate Sharpe ratio, max drawdown, and other metrics

models={
    'A2C': df_account_value_a2c,
    'DDPG': df_account_value_ddpg,
    'PPO': df_account_value_ppo,
    'TD3': df_account_value_td3,
    'SAC': df_account_value_sac
}
actions={
    'A2C': df_actions_a2c,
    'DDPG': df_actions_ddpg,
    'PPO': df_actions_ppo,
    'TD3': df_actions_td3,
    'SAC': df_actions_sac
}

metrics = {}

for model_name, df_account_value in models.items():
    if df_account_value is None:
        continue
    df_actions = actions[model_name]
    # print(f"Model: {model_name}", actions[model_name])
    df_actions.to_csv(f"actions_{model_name}.csv")
    df_account_value['daily_return'] = df_account_value.account_value.pct_change(1)
    sharpe = (252**0.5) * df_account_value['daily_return'].mean() / df_account_value['daily_return'].std()
    running_max = df_account_value['account_value'].cummax()
    drawdown = (df_account_value['account_value'] - running_max) / running_max
    max_dd = drawdown.min()
    cumulative_return = df_account_value['account_value'].iloc[-1] / df_account_value['account_value'].iloc[0] - 1
    volatility = df_account_value['daily_return'].std() * (252**0.5)
    
    metrics[model_name] = {
        "Sharpe Ratio": sharpe,
        "Max Drawdown": max_dd,
        "Cumulative Return": cumulative_return,
        "Volatility": volatility
    }
    
    print(f"Model: {model_name}")
    print(f"Sharpe Ratio: {sharpe:.2f}")
    print(f"Max Drawdown: {max_dd:.2f}")
    print(f"Cumulative Return: {cumulative_return:.2f}")
    print(f"Volatility: {volatility:.2f}")
    
latex_table = "\\begin{table}[H]\n\\centering\n\\begin{tabular}{|l|l|l|l|l|}\n\\hline\n"
latex_table += "Model & Sharpe Ratio & Max Drawdown & Cumulative Return & Volatility \\\\ \\hline\n"

for model_name, model_metrics in metrics.items():
    latex_table += f"{model_name} & {model_metrics['Sharpe Ratio']:.2f} & {model_metrics['Max Drawdown']:.2f} & {model_metrics['Cumulative Return']:.2f} & {model_metrics['Volatility']:.2f} \\\\ \\hline\n"

latex_table += f"\\end{{tabular}}\n\\caption{{Performance Metrics of Different Models under {EXP_NAME[exp]}}}\n\\label{{tab:metrics}}\n\\end{{table}}"

with open(EXP_NAME[exp]+TRAINED_MODEL_DIR + "/metrics.tex", "w") as f:
    f.write(latex_table)
    

# Part 3: Mean Variance Optimization

Mean Variance optimization is a very classic strategy in portfolio management. Here, we go through the whole process to do the mean variance optimization and add it as a baseline to compare.

First, process dataframe to the form for MVO weight calculation.

In [14]:
def process_df_for_mvo(df):
  return df.pivot(index="date", columns="tic", values="close")

### Helper functions for mean returns and variance-covariance matrix

In [15]:
# Codes in this section partially refer to Dr G A Vijayalakshmi Pai

# https://www.kaggle.com/code/vijipai/lesson-5-mean-variance-optimization-of-portfolios/notebook

def StockReturnsComputing(StockPrice, Rows, Columns): 
  import numpy as np 
  StockReturn = np.zeros([Rows-1, Columns]) 
  for j in range(Columns):        # j: Assets 
    for i in range(Rows-1):     # i: Daily Prices 
      StockReturn[i,j]=((StockPrice[i+1, j]-StockPrice[i,j])/StockPrice[i,j])* 100 
      
  return StockReturn

### Calculate the weights for mean-variance

In [ ]:
StockData = process_df_for_mvo(train)
TradeData = process_df_for_mvo(trade)

TradeData.to_numpy()

In [17]:
#compute asset returns
arStockPrices = np.asarray(StockData)
[Rows, Cols]=arStockPrices.shape
arReturns = StockReturnsComputing(arStockPrices, Rows, Cols)

#compute mean returns and variance covariance matrix of returns
meanReturns = np.mean(arReturns, axis = 0)
covReturns = np.cov(arReturns, rowvar=False)
 
#set precision for printing results
np.set_printoptions(precision=3, suppress = True)

#display mean returns and variance-covariance matrix of returns
# print('Mean returns of assets in k-portfolio 1\n', meanReturns)
# print('Variance-Covariance matrix of returns\n', covReturns)

### Use PyPortfolioOpt

In [18]:
from pypfopt.efficient_frontier import EfficientFrontier

ef_mean = EfficientFrontier(meanReturns, covReturns, weight_bounds=(0, 0.5))
raw_weights_mean = ef_mean.max_sharpe()
cleaned_weights_mean = ef_mean.clean_weights()
mvo_weights = np.array([1000000 * cleaned_weights_mean[i] for i in range(len(cleaned_weights_mean))])
# mvo_weights

In [ ]:
LastPrice = np.array([1/p for p in StockData.tail(1).to_numpy()[0]])
Initial_Portfolio = np.multiply(mvo_weights, LastPrice)
Initial_Portfolio

In [ ]:
Portfolio_Assets = TradeData @ Initial_Portfolio
# print(TradeData)
print (Initial_Portfolio)
MVO_result = pd.DataFrame(Portfolio_Assets, columns=["Mean Var"])
MVO_result
# Dividing by 1st days price to get the return
MVO_result = (MVO_result / MVO_result.iloc[0]) * 1000000
print(MVO_result)

# Part 4: DJIA index

Add DJIA index as a baseline to compare with.

In [ ]:
df_dji = YahooDownloader(
    start_date=TRADE_START_DATE, end_date=TRADE_END_DATE, ticker_list=["^DJI"]  
).fetch_data()

In [22]:
df_dji = df_dji[["date", "close"]]
fst_day = df_dji["close"][0]
dji = pd.merge(
    df_dji["date"],
    df_dji["close"].div(fst_day).mul(1000000),
    how="outer",
    left_index=True,
    right_index=True,
).set_index("date")

<a id='4'></a>
# Part 5: Backtesting Results
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

In [23]:
df_result_a2c = (
    df_account_value_a2c.set_index(df_account_value_a2c.columns[0])
    if if_using_a2c
    else None
)
df_result_ddpg = (
    df_account_value_ddpg.set_index(df_account_value_ddpg.columns[0])
    if if_using_ddpg
    else None
)
df_result_ppo = (
    df_account_value_ppo.set_index(df_account_value_ppo.columns[0])
    if if_using_ppo
    else None
)
df_result_td3 = (
    df_account_value_td3.set_index(df_account_value_td3.columns[0])
    if if_using_td3
    else None
)
df_result_sac = (
    df_account_value_sac.set_index(df_account_value_sac.columns[0])
    if if_using_sac
    else None
)

result = pd.DataFrame(
    {
        "a2c": df_result_a2c["account_value"] if if_using_a2c else None,
        "ddpg": df_result_ddpg["account_value"] if if_using_ddpg else None,
        "ppo": df_result_ppo["account_value"] if if_using_ppo else None,
        "td3": df_result_td3["account_value"] if if_using_td3 else None,
        "sac": df_result_sac["account_value"] if if_using_sac else None,
        "mvo": MVO_result["Mean Var"],
        "dji": dji["close"],
    }
)

In [ ]:
result

Now, everything is ready, we can plot the backtest result.

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)
plt.figure()
result.plot()
plt.savefig(EXP_NAME[exp]+TRAINED_MODEL_DIR + "/result.png")